In [146]:
import pandas as pd
from datetime import date, timedelta , datetime
import warnings
warnings.filterwarnings('ignore')


## FUNCTION DEFINITIONS 


In [147]:
def set_period(p):
    period_end=date(2022,10,31)
    period_start=period_end - timedelta(days=p-1)
    return ( period_start , period_end)


In [148]:
set_period(29)


(datetime.date(2022, 10, 3), datetime.date(2022, 10, 31))

In [149]:
def is_based(df,period_start , period_end):
    for i in range(len(df)):
        if None:
            pass 
        if df['status'][i]==3:
            if df['START_DATE'][i].date()<= period_start <= df['END_DATE'][i].date():
                return (True , i)
        else:
            df['START_DATE'][i].date()<= period_start
            return (True, i)

    return (False , None)

In [150]:
def is_new(df,period_start , period_end ):
    for i in range(len(df)):
        if period_start <= df['START_DATE'].min().date()<=period_end:
            return (True , 0)
    return (False , None)

In [151]:
is_based(df,date(2022, 10, 3) , date(2022, 10, 31))

(True, 0)

In [152]:
def is_churned(df,period_start , period_end ):
    for i in range(len(df)):

        if pd.isnull(df['END_DATE'][i]):
            continue 
        if period_start<= df['END_DATE'][i].date()<= period_end:
            return (True , i)
    return(False , None)

In [153]:
def is_resurrected(df,period_start , period_end ):
    for i in range(1,len(df)):
        if period_start <= df['START_DATE'][i].date()<=period_end:
            return (True , i)
    return (False , None)

In [154]:
def generate_user_tags(df, period = 30, period_start = None, period_end = None):
    
    if period_start is None and period_end is None:
        period_start, period_end = set_period(period)
    
    if period_start is None:
        period_start = period_end - timedelta(days = period - 1)
    
    if period_end is None:
        period_end = period_start + timedelta(days = period - 1)
    
    
    
    r = []
    
    
    users = set(df['user_id'])
    
    for user_id in users:
        
        mask = (df['user_id'] == user_id)
        user = df[mask]
        user.reset_index(drop = True, inplace = True)
        
        
        test, index = is_based(user, period_start, period_end)
        if test:
            r.append([ user['user_id'][index], user['status'].iloc[-1], user['plan'][index], "Base" ])
        
        test, index = is_new(user, period_start, period_end)
        if test:
            r.append([ user['user_id'][index], user['status'].iloc[-1], user['plan'][index], "New" ])
        
        test, index = is_churned(user, period_start, period_end)
        if test:
            r.append([ user['user_id'][index], user['status'].iloc[-1], user['plan'][index], "Churned" ])
        
        test, index = is_resurrected(user, period_start, period_end)
        if test:
            r.append([ user['user_id'][index], user['status'].iloc[-1], user['plan'][index], "Resurrected" ])
    
    
    
    result = pd.DataFrame(data = r, columns = ["user_id", "current status", "plan", "tag"])
    result.sort_values(by = "user_id", inplace = True, ignore_index = True)
    
    
    return result

## DATA EXPLORATION 

In [155]:
df=pd.read_csv("E:/ProgrammingPortfolio/DataScience/Growth-Analysis-Project/Waterfall_subscriptions.csv")
df.columns

Index(['user_id', 'subscription_id', 'plan', 'status', 'START_DATE',
       'END_DATE'],
      dtype='object')

In [156]:
df['START_DATE'] = pd.to_datetime(df['START_DATE']  , infer_datetime_format=True)

df['END_DATE'] = pd.to_datetime(df['END_DATE']  , infer_datetime_format=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9653 entries, 0 to 9652
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          9653 non-null   int64         
 1   subscription_id  9653 non-null   int64         
 2   plan             9653 non-null   object        
 3   status           9653 non-null   int64         
 4   START_DATE       9653 non-null   datetime64[ns]
 5   END_DATE         6081 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 452.6+ KB


## TEST 

In [157]:
df.head()

,user_id,subscription_id,plan,status,START_DATE,END_DATE
0,31,8368,Annual,1,2022-03-31,NaT
1,182,8246,Monthly,3,2022-03-26,2022-06-08
2,182,9415,Monthly,3,2022-07-13,2022-08-13
3,221,4629,Monthly,3,2021-06-10,2021-07-10
4,235,6429,Annual,1,2021-11-28,NaT


In [158]:

test=generate_user_tags(df)
test.head()

,user_id,current status,plan,tag
0,31,1,Annual,Base
1,235,1,Annual,Base
2,591,1,Annual,Base
3,616,1,Annual,Base
4,618,1,Annual,Base


## DATA EXPORT

In [159]:
result_last_30=generate_user_tags(df,period=30)
result_last_90=generate_user_tags(df,period=90)
result_last_365=generate_user_tags(df,period=365)

In [160]:
result_last_30.to_csv('Waterfall_subs_last_30.csv', index = False)
result_last_90.to_csv('Waterfall_subs_last_90.csv', index = False)
result_last_365.to_csv('Waterfall_subs_last_365.csv', index = False)